In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        # 定义第一个卷积层，输入通道1（例如灰度图像），输出通道32，卷积核大小3x3
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        
        # 定义第二个卷积层，输入通道32，输出通道64，卷积核大小3x3
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        
        # 定义第三个卷积层，输入通道64，输出通道128，卷积核大小3x3
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        
        # 定义全连接层
        self.fc1 = nn.Linear(128 * 8 * 8, 512)  # 假设输入图像大小为 32x32
        self.fc2 = nn.Linear(512, 10)  # 输出10个类
        
        # dropout层
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # 第一个卷积层 + ReLU + 最大池化
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        
        # 第二个卷积层 + ReLU + 最大池化
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        
        # 第三个卷积层 + ReLU + 最大池化
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2)
        
        # 展平多维输入数据为一维
        x = x.view(-1, 128 * 8 * 8)
        
        # 全连接层 + dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        
        # 最后一层输出
        x = self.fc2(x)
        
        return x


In [3]:
# 实例化模型
model = SimpleCNN()

# 打印模型架构
print(model)


SimpleCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=8192, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [8]:
print("模型的所有参数：")
for name, param in model.named_parameters():
    print(f"{name} - {param.size()}")

模型的所有参数：
conv1.weight - torch.Size([32, 1, 3, 3])
conv1.bias - torch.Size([32])
conv2.weight - torch.Size([64, 32, 3, 3])
conv2.bias - torch.Size([64])
conv3.weight - torch.Size([128, 64, 3, 3])
conv3.bias - torch.Size([128])
fc1.weight - torch.Size([512, 8192])
fc1.bias - torch.Size([512])
fc2.weight - torch.Size([10, 512])
fc2.bias - torch.Size([10])


In [34]:
def H(model: torch.nn.Module, L: int):
    blocks_name = set(n.split(".")[0] for n, _ in model.named_parameters())
    layers_name = []
    
    for n, _ in model.named_parameters():
        name = n.split(".")[0]
        if name in blocks_name and name not in layers_name:
            layers_name.append(name)
    
    layers_num = len(layers_name)
    
    if layers_num < L:
        raise RuntimeError(f"layers_num < L({L})")
    
    based_layers_name = layers_name[:layers_num - L]
    personalized_layers_name = layers_name[layers_num - L:]
    
    print(based_layers_name)
    print(personalized_layers_name)
    
    
    
H(model, 3)

['conv1', 'conv2']
['conv3', 'fc1', 'fc2']


In [38]:
for (k, p1), p0 in zip( model.named_parameters(),model.parameters()):
    print(k, p1.shape, p0.shape)

conv1.weight torch.Size([32, 1, 3, 3]) torch.Size([32, 1, 3, 3])
conv1.bias torch.Size([32]) torch.Size([32])
conv2.weight torch.Size([64, 32, 3, 3]) torch.Size([64, 32, 3, 3])
conv2.bias torch.Size([64]) torch.Size([64])
conv3.weight torch.Size([128, 64, 3, 3]) torch.Size([128, 64, 3, 3])
conv3.bias torch.Size([128]) torch.Size([128])
fc1.weight torch.Size([512, 8192]) torch.Size([512, 8192])
fc1.bias torch.Size([512]) torch.Size([512])
fc2.weight torch.Size([10, 512]) torch.Size([10, 512])
fc2.bias torch.Size([10]) torch.Size([10])


In [43]:
a = [1,2,3,4,5,6]
b = a[:2] 
b = b + a[3:]
b

[1, 2, 4, 5, 6]

In [48]:
a = [torch.tensor(1), torch.tensor(2), torch.tensor(3)]

b = torch.tensor(a)
a, b
b + b / 3

tensor([1.3333, 2.6667, 4.0000])